In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
img = cv2.imread('/content/drive/MyDrive/chest/COVID-19_Radiography_Dataset/Normal/masks/Normal-1.png')
img.shape

(256, 256, 3)

In [8]:
import os
import numpy as np
import cv2
from tqdm import tqdm

image_path = '/content/drive/MyDrive/chest/COVID-19_Radiography_Dataset'

images = []
masks = []

img_size = (192, 192, 3)

categories = sorted(os.listdir(image_path))
if '.ipynb_checkpoints' in categories:
    categories.remove('.ipynb_checkpoints')
num_categories = len(categories)

total_files = 0
for category in categories:
    image_folder = os.path.join(image_path, category, 'images')
    total_files += len([f for f in os.listdir(image_folder) if f.endswith('.png') or f.endswith('.jpg')])

#tqdm을 사용하여 전체 진행 상태 표시
with tqdm(total=total_files, desc="Overall Progress") as pbar:
    for category_index, category in enumerate(categories):
        image_folder = os.path.join(image_path, category, 'images')
        mask_folder = os.path.join(image_path, category, 'masks')

        for image_name in os.listdir(image_folder):
            if image_name.endswith('.png'):
                img_full_path = os.path.join(image_folder, image_name)
                mask_full_path = os.path.join(mask_folder, image_name)

                img = cv2.imread(img_full_path)
                img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                mask = cv2.imread(mask_full_path, 0)
                mask = np.where(mask > 0, category_index, mask)

                images.append(img_cvt)
                masks.append(mask)

                pbar.update(1)

Overall Progress: 100%|██████████| 21165/21165 [08:34<00:00, 41.11it/s]


In [9]:
from sklearn.model_selection import train_test_split
X_train , X_val , y_train , y_val = train_test_split(images , masks , test_size = 0.2)

In [10]:
len(X_train)

16932

In [11]:
import tensorflow as tf
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(factor=0.05),
    tf.keras.layers.RandomFlip(mode='horizontal')
])

In [12]:
import tensorflow as tf
def preprocessing_data(image , label):
  image = tf.image.resize(image , img_size[:2])
  label = tf.image.resize(label[... , tf.newaxis] , img_size[:2])
  return image , label

def augment_image(image , label):
  image = data_augmentation(image)
  return image , label

def create_tensorflow_dataset(images , labels , batch_size = 32 , buffer_size = 1000 , augment = False):
  dataset = tf.data.Dataset.from_tensor_slices((images , labels))
  dataset = dataset.map(preprocessing_data , num_parallel_calls = tf.data.AUTOTUNE)
  if augment:
    dataset = dataset.map(augment_image , num_parallel_calls = tf.data.AUTOTUNE)
  dataset = dataset.shuffle(buffer_size)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size = tf.data.AUTOTUNE)
  return dataset

In [13]:
train_dataset = create_tensorflow_dataset(X_train, y_train, batch_size=16, buffer_size=len(X_train), augment= True)
val_dataset = create_tensorflow_dataset(X_val, y_val, batch_size=16, buffer_size=len(X_val), augment=False)


In [ ]:
import tensorflow as tf

def unet_model(input_size=(192, 192, 3), num_classes= 4, dropout_rate=0.3):
    inputs = tf.keras.layers.Input(input_size)

    c1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(inputs)
    c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Activation('relu')(c1)
    c1 = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(c1)
    c1 = tf.keras.layers.BatchNormalization()(c1)
    c1 = tf.keras.layers.Activation('relu')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)
    p1 = tf.keras.layers.Dropout(dropout_rate)(p1)

    c2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(p1)
    c2 = tf.keras.layers.BatchNormalization()(c2)
    c2 = tf.keras.layers.Activation('relu')(c2)
    c2 = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(c2)
    c2 = tf.keras.layers.BatchNormalization()(c2)
    c2 = tf.keras.layers.Activation('relu')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
    p2 = tf.keras.layers.Dropout(dropout_rate)(p2)

    c3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(p2)
    c3 = tf.keras.layers.BatchNormalization()(c3)
    c3 = tf.keras.layers.Activation('relu')(c3)
    c3 = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(c3)
    c3 = tf.keras.layers.BatchNormalization()(c3)
    c3 = tf.keras.layers.Activation('relu')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
    p3 = tf.keras.layers.Dropout(dropout_rate)(p3)

    c4 = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(p3)
    c4 = tf.keras.layers.BatchNormalization()(c4)
    c4 = tf.keras.layers.Activation('relu')(c4)
    c4 = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(c4)
    c4 = tf.keras.layers.BatchNormalization()(c4)
    c4 = tf.keras.layers.Activation('relu')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)
    p4 = tf.keras.layers.Dropout(dropout_rate)(p4)

    c5 = tf.keras.layers.Conv2D(512, (3, 3), padding='same')(p4)
    c5 = tf.keras.layers.BatchNormalization()(c5)
    c5 = tf.keras.layers.Activation('relu')(c5)
    c5 = tf.keras.layers.Conv2D(512, (3, 3), padding='same')(c5)
    c5 = tf.keras.layers.BatchNormalization()(c5)
    c5 = tf.keras.layers.Activation('relu')(c5)

    # Decoder
    u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4], axis=-1)
    c6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(u6)
    c6 = tf.keras.layers.BatchNormalization()(c6)
    c6 = tf.keras.layers.Activation('relu')(c6)
    c6 = tf.keras.layers.Conv2D(256, (3, 3), padding='same')(c6)
    c6 = tf.keras.layers.BatchNormalization()(c6)
    c6 = tf.keras.layers.Activation('relu')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3], axis=-1)
    c7 = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(u7)
    c7 = tf.keras.layers.BatchNormalization()(c7)
    c7 = tf.keras.layers.Activation('relu')(c7)
    c7 = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(c7)
    c7 = tf.keras.layers.BatchNormalization()(c7)
    c7 = tf.keras.layers.Activation('relu')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2], axis=-1)
    c8 = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(u8)
    c8 = tf.keras.layers.BatchNormalization()(c8)
    c8 = tf.keras.layers.Activation('relu')(c8)
    c8 = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(c8)
    c8 = tf.keras.layers.BatchNormalization()(c8)
    c8 = tf.keras.layers.Activation('relu')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=-1)
    c9 = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(u9)
    c9 = tf.keras.layers.BatchNormalization()(c9)
    c9 = tf.keras.layers.Activation('relu')(c9)
    c9 = tf.keras.layers.Conv2D(32, (3, 3), padding='same')(c9)
    c9 = tf.keras.layers.BatchNormalization()(c9)
    c9 = tf.keras.layers.Activation('relu')(c9)

    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(c9)

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model


In [ ]:
model = unet_model(input_size = img_size, dropout_rate=0.3)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss' , patience = 10)
histroy = model.fit(train_dataset , validation_data = val_dataset , epochs = 50 , callbacks = [early_stopping_cb])

In [11]:
model.save('/content/drive/MyDrive/chest/my_covid_image_segmentation.keras')

In [12]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 192, 192, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 192, 192, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 192, 192, 32)   │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 192, 192, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 192, 192, 32)   │          9,248 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 192, 192, 32)   │            128 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 192, 192, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 96, 96, 32)     │              0 │ activation_1[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 96, 96, 32)     │              0 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 96, 96, 64)     │         18,496 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 96, 96, 64)     │            256 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 96, 96, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 96, 96, 64)     │         36,928 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 96, 96, 64)     │            256 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 96, 96, 64)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 48, 48, 64)     │              0 │ activation_3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)  

 Total params: 23,304,142 (88.90 MB)

 Trainable params: 7,766,084 (29.63 MB)

 Non-trainable params: 5,888 (23.00 KB)

 Optimizer params: 15,532,170 (59.25 MB)

In [ ]:
#오리지날 이미지, 오리지날 마스크 이미지 , 예측 마스크 이미지 시각화
import matplotlib.pyplot as plt

model = tf.keras.models.load_model('/content/drive/MyDrive/chest/my_covid_image_segmentation.keras')

def visualize_prediction(model, dataset, categories, num_images=5):
    for images, true_masks in dataset.take(num_images):
        predictions = model.predict(images)
        predicted_masks = tf.argmax(predictions, axis=-1)

        for i in range(len(images)):
            category_index = tf.reduce_max(true_masks[i]).numpy().astype(int)
            category_name = categories[category_index]

            plt.figure(figsize=(15, 5))
            plt.subplot(131)
            plt.imshow(tf.cast(images[i], tf.uint8))
            plt.title(f"Original: {category_name}")
            plt.axis('off')


            plt.subplot(132)
            plt.imshow(tf.cast(true_masks[i], tf.uint8), cmap='gray')
            plt.title("True Mask")
            plt.axis('off')

            plt.subplot(133)
            plt.imshow(predicted_masks[i], cmap='gray')
            plt.title("Predicted Mask")
            plt.axis('off')

            plt.show()

visualize_prediction(model, val_dataset, categories)
